In [ ]:
from dotenv import load_dotenv
import os

load_dotenv()
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")

#### 다음 실습 코드는 학습 목적으로만 사용 바랍니다. 문의 : audit@korea.ac.kr 임성열 Ph.D.

#### 1. 노드 내부에서 툴 선택 : 각 노드를 함수처럼 정의하고, 그 안에서 필요한 툴을 직접 호출하도록 코드를 작성

In [ ]:
!pip install langchain langgraph langchain-openai

In [ ]:
# 환경변수 가져오기
# openai_api_key = os.getenv("OPENAI_API_KEY")
# serpapi_key = os.getenv("SERPAPI_API_KEY")

# 또는 다음과 같이 직접 키 입력 (개발)
# os.environ["OPENAI_API_KEY"] = ""  # 자신의 OpenAI 키
# os.environ["SERPAPI_API_KEY"] = "9c61eeb3bac82aea465b08d257b1a419dbf02d1734808b80cae1e881ae796196"

In [70]:
# LLM 설정 (예: gpt-4)
llm = ChatOpenAI(model="gpt-4", temperature=0)

In [71]:
# 기본 임포트
from typing import TypedDict, Literal
import re, ast, operator, hashlib
from langgraph.graph import StateGraph

In [72]:
# 상태 정의
class State(TypedDict, total=False):
    query: str
    route: Literal["web", "math"]
    result: str

In [73]:
def is_math_query(query: str) -> bool:
    """
    단순하게 수학 연산 여부를 판별하는 함수.
    숫자와 사칙연산 기호만 포함되면 True 반환.
    """
    return bool(re.fullmatch(r"[0-9+\-*/(). ]+", query.strip()))

In [74]:
# 유틸리티 함수 (수학식 판별)
_ARITH_RE = re.compile(r"^\s*[-+*/\d\s().]+\s*$")

def is_math_query(q: str) -> bool:
    return bool(_ARITH_RE.match(q))

In [76]:
# 노드 구현 (웹 검색 / 수학 계산)
# 웹 검색 노드 (여기서는 더미 함수로 대체)
def web_scraper(query: str, num_results: int = 5) -> str:
    """구글에서 검색어에 대한 상위 요약 검색 결과를 반환합니다."""
    try:
        serpapi_key = os.getenv("SERPAPI_API_KEY")
        params = {
            "engine": "google",
            "q": query,
            "hl": "ko",
            "gl": "kr",
            "api_key": serpapi_key
        }

        search = GoogleSearch(params)
        results = search.get_dict()

        if "organic_results" not in results or not results["organic_results"]:
            return "검색 결과 없음"

        titles = [res.get("title", "제목 없음") for res in results["organic_results"][:num_results]]
        return "\n".join(f"- {title}" for title in titles)
    except Exception as e:
        return f"웹 검색 오류: {str(e)}"

# web_node는 web_scraper 툴만 실행하도록 고정됨
def web_node(state: State) -> State:
    q = state["query"]
    try:
        summary = web_scraper(q) # web_node는 무조건 web_scraper만 호출
    except Exception as e:
        summary = f"웹 검색 오류: {e}"
    return {"result": f"[웹검색 요약]\n{summary}"}


# 수학 계산 노드
_ops = {
    ast.Add: operator.add, ast.Sub: operator.sub,
    ast.Mult: operator.mul, ast.Div: operator.truediv,
    ast.USub: operator.neg, ast.UAdd: operator.pos,
    ast.Pow: operator.pow,
}

def _eval_expr(node):
    if isinstance(node, ast.Num):
        return node.n
    if isinstance(node, ast.Constant) and isinstance(node.value, (int, float)):
        return node.value
    if isinstance(node, ast.BinOp) and type(node.op) in _ops:
        return _ops[type(node.op)](_eval_expr(node.left), _eval_expr(node.right))
    if isinstance(node, ast.UnaryOp) and type(node.op) in _ops:
        return _ops[type(node.op)](_eval_expr(node.operand))
    raise ValueError("허용되지 않은 연산입니다.")

# math_node는 산술 계산 툴(_eval_expr)만 실행하도록 고정됨
def math_node(state: State) -> State:
    q = state["query"]
    try:
        node = ast.parse(q, mode="eval").body
        val = _eval_expr(node) # math_node는 무조건 _eval_expr만 호출
        return {"result": f"[계산 결과] {q} = {val}"}
    except Exception:
        return {"result": "유효한 산술식이 아닙니다."}

In [77]:
# 라우터 노드 (질문에 따라 web/math 분기)
def router(state: State) -> State:
    q = state["query"]
    route: Literal["web","math"] = "math" if is_math_query(q) else "web"
    return {"route": route}

In [78]:
# 그래프 빌드
def build_graph():
    builder = StateGraph(State)

    builder.add_node("router", router)
    builder.add_node("web_node", web_node)
    builder.add_node("math_node", math_node)

    # router → 조건 분기
    builder.add_conditional_edges(
        "router",
        lambda s: s["route"],
        {
            "web": "web_node",
            "math": "math_node",
        },
    )

    builder.set_entry_point("router")
    return builder.compile()

graph = build_graph()

In [79]:
# 실행 테스트
print(graph.invoke({"query": "3 + 5"}))            # 수학 → math_node
print(graph.invoke({"query": "LangGraph 사용법"})) # 일반 질문 → web_node

{'query': '3 + 5', 'route': 'math', 'result': '[계산 결과] 3 + 5 = 8'}
{'query': 'LangGraph 사용법', 'route': 'web', 'result': '[웹검색 요약]\n- Part 1. 랭그래프 LangGraph 기초\n- LangGraph) LangGraph에 대한 개념과 간단한 예시 만들어보기\n- 1-2. LangGraph 환경 설정 및 기본 사용법\n- 랭그래프(LangGraph)란? LangGraph의 개념과 사용 방법 예제 ...\n- LangGraph Retrieval Agent를 활용한 동적 문서 검색 및 처리'}


##### 2. AgentExecutor + Tool Routing : LangChain 스타일로, 노드 안에서 에이전트를 실행하고, 에이전트에 어떤 툴 세트를 줄지 지정

In [63]:
# 필요한 패키지 임포트
from langchain.agents import create_openai_functions_agent, AgentExecutor
from langchain.prompts import ChatPromptTemplate, MessagesPlaceholder

# 공통 프롬프트 템플릿 정의
base_prompt = ChatPromptTemplate.from_messages([
    ("system", "당신은 주어진 도구만 사용할 수 있는 유능한 AI 어시스턴트입니다."),
    MessagesPlaceholder("chat_history"),
    ("user", "{input}"),
    MessagesPlaceholder("agent_scratchpad"),
])

In [65]:
# 예시: 웹 검색 툴
from langchain.tools import tool
# SerpAPI용 라이브러리
from serpapi import GoogleSearch

@tool
def web_scraper(query: str, num_results: int = 5) -> str:
    """구글에서 검색어에 대한 상위 요약 검색 결과를 반환합니다."""
    try:
        serpapi_key = os.getenv("SERPAPI_API_KEY")
        params = {
            "engine": "google",
            "q": query,
            "hl": "ko",
            "gl": "kr",
            "api_key": serpapi_key
        }

        search = GoogleSearch(params)
        results = search.get_dict()

        if "organic_results" not in results or not results["organic_results"]:
            return "검색 결과 없음"

        titles = [res.get("title", "제목 없음") for res in results["organic_results"][:num_results]]
        return "\n".join(f"- {title}" for title in titles)
    except Exception as e:
        return f"웹 검색 오류: {str(e)}"

# 예시: 수학 덧셈 툴
@tool
def calc_add(x: int, y: int) -> int:
    """두 숫자를 더한다."""
    return x + y

# 🔹 각각 툴들을 리스트로 묶어줌
tools_web = [web_scraper]
tools_math = [calc_add]

In [66]:
# 웹검색 AgentExecutor
agent_web = create_openai_functions_agent(
    llm=llm,
    tools=tools_web, # 웹검색 Tool : web_scraper을 쓰도록 지정
    prompt=base_prompt
)
exec_web = AgentExecutor(agent=agent_web, tools=tools_web, verbose=True)

In [67]:
# 계산 AgentExecutor
agent_math = create_openai_functions_agent(
    llm=llm,
    tools=tools_math, # 산술연산 Tool : calc_add을 쓰도록 지정
    prompt=base_prompt
)
exec_math = AgentExecutor(agent=agent_math, tools=tools_math, verbose=True)

In [43]:
# 테스트 실행
print(exec_web.invoke({"input": "LangChain을 검색해서 요약하라", "chat_history": []}))
print(exec_math.invoke({"input": "3과 5를 더하라", "chat_history": []}))




> Entering new AgentExecutor chain...

Invoking: `web_scraper` with `{'query': 'LangChain', 'num_results': 5}`


- LangChain
- 랭체인 LangChain 이란 무엇인가? | 인사이트리포트
- LangChain이란 무엇인가요?
- langchain-ai/langchain: 🦜🔗 Build context-aware reasoning ...
- Part 1. LangChain 기초LangChain은 AI 모델을 활용하여 다양한 애플리케이션을 개발할 수 있도록 돕는 프레임워크입니다. 이 프레임워크는 고급 언어 모델을 사용하여 개발자들이 명령의 맥락을 이해하고, 그에 따라 반응할 수 있도록 지원합니다. LangChain은 대화형 응용 프로그램, 데이터 분석 및 자연어 처리(NLP) 작업에 적합하며, 여러 기능을 통합하여 사용자 맞춤형 솔루션을 제공합니다. 

특히 LangChain은 다음과 같은 주요 특징을 갖고 있습니다:

1. **맥락 인식**: 언어 모델이 이전 대화나 명령의 맥락을 파악하여 더 의미 있는 반응을 생성합니다.
2. **모듈형 구조**: 개발자가 쉽게 기능을 추가하거나 변경할 수 있는 구조를 가지고 있어 유연성이 높습니다.
3. **다양한 사용 사례**: 대화형 챗봇, 사용자 데이터 분석, 콘텐츠 생성 등 다양한 분야에서 활용할 수 있습니다.

LangChain은 AI와 언어 처리의 통합을 통해 개발자와 기업이 더 효율적으로 작업할 수 있도록 돕는 혁신적인 도구로 자리 잡고 있습니다.

> Finished chain.
{'input': 'LangChain을 검색해서 요약하라', 'chat_history': [], 'output': 'LangChain은 AI 모델을 활용하여 다양한 애플리케이션을 개발할 수 있도록 돕는 프레임워크입니다. 이 프레임워크는 고급 언어 모델을 사용하여 개발자들이 명령의 맥락을 이해하고, 그에 따라 반응할 수 있도록 지원합니다